In [1]:
!pip install ultralytics
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import torch
from ultralytics import YOLO
import glob, shutil, os
import wandb

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/archive (1).zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
# Load a pretrained YOLO11n model
model = YOLO("yolo11m.pt")

In [ ]:
!wandb login

In [ ]:
# W&B Sweep 설정을 위한 Python 딕셔너리
sweep_config = {
    'method': 'bayes',  # 이전 결과를 바탕으로 다음 파라미터를 추천하는 베이지안 방식
    'metric': {
        'name': 'metrics/mAP50-95',  # W&B에 실제 기록되는 mAP50-95 지표 이름으로 수정해주세요.
        'goal': 'maximize'          # 이 지표를 최대화하는 것이 목표
    },
    'parameters': {
        # --- 분석을 통해 성능이 검증된 값으로 고정 ---
        'epochs': {
            'value': 50
        },
        'batch': {
            'value': 32  # 가장 성능이 좋았던 배치 사이즈 32로 고정
        },
        'optimizer': {
            'value': 'SGD'  # AdamW 대비 월등한 성능을 보인 SGD로 고정
        },

        # --- 최적점을 찾기 위해 탐색할 파라미터 범위 ---
        'lr0': {
            'distribution': 'uniform',
            'min': 0.0025,
            'max': 0.0055  # 성공적이었던 1번, 5번 실험의 lr0 값을 중심으로 탐색
        },
        'momentum': {
            'distribution': 'uniform',
            'min': 0.85,
            'max': 0.94  # 좋은 성능을 보인 실험들의 모멘텀 값(0.85 ~ 0.92) 주변을 집중 탐색
        },
        'weight_decay': {
            'distribution': 'uniform',
            'min': 0.0005,
            'max': 0.00095 # 가장 좋았던 1번, 5번 실험의 값 주변을 탐색
        }
    }
}

# W&B 프로젝트에 Sweep을 생성하고, 그 ID를 받아옵니다.
sweep_id = wandb.sweep(sweep_config, project="yolo_total_data")


In [ ]:
import os
import shutil
import wandb  # 👈 [개선 1] wandb 라이브러리 import 추가
from ultralytics import YOLO

def train_function():
    run = None  # run 객체를 먼저 초기화
    try:
        # 1. W&B 실행 초기화
        # 프로젝트와 엔티티를 명시해주는 것이 좋습니다.
        run = wandb.init()

        # YOLO 모델 로드 ("yolov8m.pt"가 일반적인 이름입니다. 오타가 아닌지 확인해보세요)
        model = YOLO("yolo11m.pt")

        # 2. model.train에 wandb.config 값 사용하기
        # model.train()은 학습 정보를 담은 results 객체를 반환합니다.
        results = model.train(
            data="/content/data/data.yaml",
            imgsz=640,
            cache=True,
            pretrained=True,
            patience=20,
            close_mosaic=10,
            name=run.name,  # 👈 [개선 2] W&B 실행 이름을 실험 폴더 이름으로 지정 (강력 추천)
            # --- W&B Sweep으로 제어되는 하이퍼파라미터 ---
            epochs=wandb.config.epochs,
            batch=wandb.config.batch,
            optimizer=wandb.config.optimizer,
            lr0=wandb.config.lr0,
            momentum=wandb.config.momentum,
            weight_decay=wandb.config.weight_decay
        )

        # 3. 학습 결과에서 직접 경로를 가져와 백업 (더 안정적인 방법)
        # 👈 [개선 3] glob 대신 results 객체를 사용해 정확한 경로 참조
        weights_dir = os.path.join(results.save_dir, 'weights')
        best_pt_path = os.path.join(weights_dir, "best.pt")

        if os.path.exists(best_pt_path):
            backup_dir = "/content/drive/MyDrive/yolo_backup"
            os.makedirs(backup_dir, exist_ok=True)
            backup_filename = f"best_{run.name}.pt"
            shutil.copy(best_pt_path, os.path.join(backup_dir, backup_filename))
            print(f"✅ best.pt 백업 완료 → {backup_dir}/{backup_filename}")

    except Exception as e:
        print(f"🚨 학습 중 오류 발생: {e}")
    finally:
        # 4. W&B 실행 종료 (오류 발생 시에도 실행 보장)
        # 👈 [개선 4] run이 성공적으로 초기화되었을 경우에만 종료
        if run:
            wandb.finish()
            print("wandb run을 성공적으로 종료했습니다.")

In [ ]:
# 에이전트 실행: 위에서 만든 train_function을 5번 실행하며 최적의 파라미터를 찾습니다.
wandb.agent(sweep_id, function=train_function, count=10)

# Sweep이 끝나면 wandb 종료
wandb.finish()